# Processing cellpy batch - life

### `{{cookiecutter.project_name}}::{{cookiecutter.session_id}}`

**Experimental-id:** `{{cookiecutter.notebook_name}}`  
**Short-name:** `{{cookiecutter.session_id}}`  
**Project:** `{{cookiecutter.project_name}}`  
**By:** `{{cookiecutter.author_name}}`   
**Date:** `{{cookiecutter.date}}`  

## Imports and setup

Note! This template was made for `cellpy` version 0.4.1.a1

In [ ]:
import os
import pathlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import holoviews as hv
from holoviews.core.io import Pickler, Unpickler
import hvplot.pandas
from holoviews import opts

import cellpy
from cellpy import prms
from cellpy import prmreader
from cellpy.utils import batch, helpers, plotutils

In [ ]:
%matplotlib inline
hv.extension('bokeh')
pd.set_option('display.max_columns', 70)
print(f"cellpy version: {cellpy.__version__}")

In [ ]:
cellpy_batch_file = "cellpy_batch_" + "{{cookiecutter.session_id}}" + ".json"

In [ ]:
b = batch.from_journal(cellpy_batch_file)

If you would like to only focus on a sub-set of the cells, you should
modify your journal pages here before linking it to the cellpy-data files.

For example, you can select only the five last cells like this:
```python
b.pages = b.pages.iloc[-5:]
```

Remember, the journal pages is a `pandas.DataFrame` so you have tons of options for tweaking it.
It is recommended that you do a linking (`b.link()` or `b.update()`) after removing or adding cells to the journal.

In [ ]:
# b.pages = b.pages.iloc[-5:]

### Linking

In [ ]:
b.link()

# or b.update() if you would like to get new data from the tester.

cell_labels = b.experiment.cell_names
print(" cell labels: ")
cell_labels

## Preprosessing summaries

You might want to add some code to edit the summaries before
processing them. You usuall dont know what you will have to edit before
you have done (at least parts of) the analysis. However, I advice that you always clean up your notebooks and put your pre-processing routines here so that it is easy to re-run the notebook (and understand it) later.

A good way of doing this is to create a yanked batch object (using `helpers.yank_outliers` or `helpers.yank_after`).

In [ ]:
# # example of dictionary used for input to yanking (the lists contains cycle-numbers that should be removed)
# bad_cycle_numbers = {
#     'cell_label_01':[4, 337, 338],
#     'cell_label_01': [4, 336, 533],
#     'cell_label_01': [4, 500],
#     'cell_label_01':[4],
#  }

In [ ]:
# # example on how to create a "yanked" batch object:

# removed = helpers.yank_outliers(b,
#     zscore_limit=None,
#     low=1000.0,
#     high=6000.0,
#     filter_cols=None,
#     freeze_indexes=None,
#     remove_indexes=bad_cycle_numbers)

# # you can also save the info about removed cycles to your journal
# b.journal.session["bad_cycles"] = removed
# b.save_journal()
# b.duplicate_journal()

In [ ]:
# # if your journal already contains information about bad cycles
# # you can yank directly from the journal.session information:
# helpers.yank_outliers(b, remove_indexes=b.journal.session["bad_cycles"])

In [ ]:
# # example on how to create a "yanked" batch object containing only cycles up to cycle 100
# b_yanked = helpers.yank_after(b, 100)

## Looking at concatenated summaries

### Example: Plotting Average Life

In [ ]:
csf = helpers.concatenate_summaries(b, columns=["charge_capacity"], group_it=True)

In [ ]:
p1 = plotutils.plot_concatenated(
    csf, 
    journal=b.experiment.journal, marker_size=5,
    extension="bokeh",
    height=500,
    spread=True,
    xlim=(0, 400),
    ylim=(0, 4000),
)
p1

In [ ]:
fig_label = "{{cookiecutter.session_id}}_charge_cap"

# save as csv:
csf.to_csv(
    f"data/processed/{fig_label}.csv", 
    sep=";", index=True,
)
# export to html:
hv.save(
    p1,
    f"out/{fig_label}.html",
    toolbar=True
)

# save as pickle (can be loaded in other notebooks)
Pickler.save(p1, f"out/{fig_label}.hvz",)

### Example: Plotting average areal capacity versus normalized capacity
In this example, only the first 1000 cycles are included.

In [ ]:
csf_area = helpers.concatenate_summaries(b, max_cycle=1000, add_areal=True, columns=["areal_charge_capacity"], normalize_cycles=True, group_it=True)

In [ ]:
p2 = plotutils.plot_concatenated(
    csf_area, 
    journal=b.experiment.journal, marker_size=5,
    extension="bokeh",
    height=500,
    spread=True,
)
p2

In [ ]:
fig_label = "{{cookiecutter.session_id}}_mean_areal_cap_norm_cycles"

# save as csv:
csf_area.to_csv(
    f"data/processed/{fig_label}.csv", 
    sep=";", index=False,
)
# export to html:
hv.save(
    p2,
    f"out/{fig_label}.html",
    toolbar=True
)

# save as pickle (can be loaded in other notebooks)
Pickler.save(p2, f"out/{fig_label}.hvz",)

### Example: Plotting Average Life for slow cycles versus normalized capacity

In [ ]:
csf_slow = helpers.concatenate_summaries(b, columns=["charge_capacity"], rate=0.2, rate_std=0.02, normalize_cycles=True, group_it=True)

In [ ]:
p3 = plotutils.plot_concatenated(
    csf_slow, 
    journal=b.experiment.journal, marker_size=5,
    extension="bokeh",
    height=500,
    spread=True,
    xlim=(0, 400),
    ylim=(1000, 4000),
)
p3

In [ ]:
fig_label = "{{cookiecutter.session_id}}_slow_mean_cap_norm_cycles"

# save as csv:
csf_slow.to_csv(
    f"data/processed/{fig_label}.csv", 
    sep=";", index=False,
)
# export to html:
hv.save(
    p3,
    f"out/{fig_label}.html",
    toolbar=True
)

# save as pickle (can be loaded in other notebooks)
Pickler.save(p3, f"out/{fig_label}.hvz",)

### Example: Simplified plotting of Average Life and Coulombic Efficiency
A convenience function combining `concatenate_summaries` and `plot_concatenated` also exists.

In [ ]:
plotutils.bplot(b, columns=["coulombic_efficiency", "charge_capacity"], group_it=True)

## Looking at individual summaries

### Example: Plotting IR

In [ ]:
cell_id = b.experiment.cell_names[0]
c = b.experiment.data[cell_id]  # you can also use "look-up" by writing b.experiment.data.x and select from the pop-up
s = c.cell.summary.reset_index()
s.head()

In [ ]:
s.hvplot(x="cycle_index", y="ir_charge_u_Ohms", xlabel="Cycle Index", label=cell_id)

## Looking at individual steps

### Example: Viewing the steps

In [ ]:
cell_id = b.experiment.cell_names[0]
c = b.experiment.data[cell_id]
t = c.cell.steps
t.head()

## Links

### Notebooks
- notes and information [link](00_{{cookiecutter.notebook_name}}_notes.ipynb)
- processing raw data [link](01_{{cookiecutter.notebook_name}}_loader.ipynb)
- life [link](02_{{cookiecutter.notebook_name}}_life.ipynb)
- cycles [link](03_{{cookiecutter.notebook_name}}_cycles.ipynb)
- ica [link](04_{{cookiecutter.notebook_name}}_ica.ipynb)
- plots [link](05_{{cookiecutter.notebook_name}}_plots.ipynb)